In [1]:
import numpy as np
import os
import time
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten

from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = 224

Loaded the images of dataset-adult

Loaded the images of dataset-child

Loaded the images of dataset-defect

Loaded the images of dataset-insect

Loaded the images of dataset-perfect

Loaded the images of dataset-reject

(2590, 224, 224, 3)


In [3]:
PATH = os.getcwd()

# Define data path
data_path ='C:/TensorFlow/vgg16/train'
data_dir_list = os.listdir(data_path)[:6] # 5 classes

img_data_list=[]

for dataset in sorted(data_dir_list):
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path + '/'+ dataset + '/'+ img 
        img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        # print('Input image shape:', x.shape)
        img_data_list.append(x)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

In [4]:

# Define the number of classes
num_classes = 6
num_of_samples = (img_data.shape[0])
labels = np.ones((num_of_samples,), dtype='int64')

labels[0:325]=0
labels[325:1100]=1
labels[1100:1400]=2
labels[1400:1525]=3
labels[1525:2325]=4
labels[2325:]=5

names = ['adult','child','defect','insect','perfect','reject']

# convert class Labels to one-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

In [5]:
image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

model = ResNet50(input_tensor=image_input, include_top=True, weights='imagenet')
model.summary()

In [6]:

# Adding classifier
last_layer = model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(num_classes, activation='softmax', name='output_layer')(x)
custom_resnet_model = Model(inputs=image_input, outputs=out)
custom_resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [10]:
# Freezing the all layers except clasifier(last-layer)
for layer in custom_resnet_model.layers[:-1]:
    layer.trainable = False

# Check whether last layer is trainable or not
custom_resnet_model.layers[-1].trainable

In [13]:
 #Training the classifier
custom_resnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

keras.engine.training.Model

In [14]:
hist = custom_resnet_model.fit(X_train, y_train, batch_size=32, epochs=1, verbose=1, validation_data=(X_test, y_test))

In [21]:
import matplotlib.pyplot as plt

In [22]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['acc']
val_acc = hist.history['val_acc']
xc = range(12)

In [23]:
plt.figure(1, figsize=(7, 5))
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train', 'val'])
plt.style.use(['classic'])

plt.figure(2, figsize=(7, 5))
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train', 'val'], loc=4)
plt.style.use(['classic'])
plt.show()

ValueError: Error when checking target: expected dense_5 to have 4 dimensions, but got array with shape (2072, 6)

In [ ]:
img_path = 'images/donut.jpg'
img = image.load_img(img_path, target_size=(IMAGE_SIZE,IMAGE_SIZE))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)

preds = custom_resnet_model2.predict(x)

print('Image: ')
plt.imshow(img)
plt.show()

print('\nPrediction: ')
preds = preds[0]
for i in range(len(preds)):
    print(names[i] + ': \t{} %'.format(preds[i]*100))

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import itertools

Y_pred = custom_resnet_model2.predict(X_test)
#print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
#print(y_pred)
#y_pred = model.predict_classes(X_test)
#print(y_pred)
target_names = ['class 0(adult)', 'class 1(child)', 'class 2(defect)','class 3(insect)', 'class 4(perfect)', 'class 5(reject)']

print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
cnf_matrix = (confusion_matrix(np.argmax(y_test,axis=1), y_pred))

np.set_printoptions(precision=2)

plt.figure()

# Plot non-normalized confusion matrix
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix')
#plt.figure()
# Plot normalized confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True,
#                      title='Normalized confusion matrix')
#plt.figure()
plt.show()

In [ ]:
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
model_json = custom_resnet_model2.to_json()
with open("model_Resnet50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
custom_resnet_model2.save_weights("model_Resnet50.h5")
print("Saved model to disk")

# load json and create model
json_file = open('model_Resnet50.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_Resnet50.h5")
print("Loaded model from disk")

custom_resnet_model2.save('model_Resnet50.hdf5')
loaded_model=load_model('model_Resnet50.hdf5')
